In [ ]:
''' Currently this code requires the use of a vpn (which also may or may not stop the "your computer may be sending 
automated queries captcha popup) and also can only book for one date that you cant even select without inspecting each 
element in the webpage individually... you must also login to your account either using the bot or manually every time 
you open a new selenium browser wchich adds time. Time delays are currently semi-random but the cpatcha still detects 
something and presents multiple captcha tests which takes more time... additionally the current code takes >50 seconds 
which is way longer than it takes for me to just manually book... also I take no responsibility if your ip address gets 
blocked for captchas lol. This code does NOT automatically run when a booking slot opens up either and must be manually
started but I think it is easier to leave this script as running in an IDE rather than to make it a .exe which would be
harder to change booking dates etc if anything was not the same between days/weeks... best use is probably to wait and 
look for when a specified booking slot gets a cancellation and then use this to book since it is so slow... or to send a 
notification that a slot is available now for the desired time/date.
'''
# also need to install chromedriver and ffmpeg and set path C:\ffmpeg\bin and install the following:
!pip install selenium
!pip install SpeechRecognition
!pip install pydub
!pip install random2
!pip install ffmpy
!pip install ffmpeg
!pip install ffprobe
!pip install fake-useragent
!pip install twilio
!pip install pause

In [8]:
# Imports
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import speech_recognition as sr
import urllib
import pydub
import random
import os
import sys
from datetime import datetime as date
from fake_useragent import UserAgent
from twilio.rest import Client # this is a free service to be able to automatically text my phone when the code either books or fails

# Variables
Num_of_members = 1 # number of people we will be booking for (bot currently can only book 1 but here for readability)
email = '000ago@gmail.com' # not currently in use
password = 'T0r1nkyl3.' # not currently in use
booking_month = 'April'
day_to_book = 'Thursday' # Notice exact formatting in day dictionary!
date_to_book = '22'
time_to_book = '9 PM to  11 PM' #be careful with format (single space between #_PM, but double space between to__#)

# Dont touch this stuff (used to determine what row and column in a calendar the desired day would be)
day_dict = {'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4, 'Friday':5, 'Saturday':6, 'Sunday':7}
today = date.today()
this_month = today.strftime("%B")
this_day = today.strftime('%A')
time_slot_dict_MTF = {'7 AM to  9 AM':1, '8 AM to  10 AM':2, '9 AM to  11 AM':3, '10 AM to  12 PM':4, '11 AM to  1 PM':5,
                     '12 PM to  2 PM':6, '1 PM to  3 PM':7, '2 PM to  4 PM':8, '3 PM to  5 PM':9, '4 PM to  6 PM':10,
                     '5 PM to  7 PM':11, '6 PM to  8 PM':12, '7 PM to  9 PM':13, '8 PM to  10 PM':14, '9 PM to  11 PM':15}
time_slot_dict_WTh = {'12 PM to  2 PM':1, '1 PM to  3 PM':2, '2 PM to  4 PM':3, '3 PM to  5 PM':4, '4 PM to  6 PM':5,
                     '5 PM to  7 PM':6, '6 PM to  8 PM':7, '7 PM to  9 PM':8, '8 PM to  10 PM':9, '9 PM to  11 PM':10}
time_slot_dict_SS = {'10 AM to  12 PM':1, '11 AM to  1 PM':2, '12 PM to  2 PM':3, '1 PM to  3 PM':4, '2 PM to  4 PM':5, 
                     '3 PM to  5 PM':6, '4 PM to  6 PM':7, '5 PM to  7 PM':8, '6 PM to  8 PM':9, '7 PM to  9 PM':10, 
                     '8 PM to  10 PM':11}

# Wait a semi-random amount of time between actions
def wait():
    i = random.randint(0,9)
    time.sleep(i/10+1)
    
    
def wait_short():
    t = random.randint(1,3)
    time.sleep(t/10)
    
    
options = Options()

options.headless = True

PATH = "C:\chromedriver.exe" # path to chromedriver
driver = webdriver.Chrome(PATH) # open automated browser window

driver.set_window_size(1920,1080) # resize window
#driver.Manage().Cookies.DeleteAllCookies()

# Determines Which Row Of The Calendar (treated like an array for indexing) Our Booking Day Is
ref_day = '01 ' + date.today().strftime('%m %Y')
day = date.strptime(ref_day, '%d %m %Y').weekday()
row = int((int(date_to_book) + int(day) - 1)/7) + 1

In [9]:
# Run this cell to login
driver.get('https://app.rockgympro.com/b/widget/?a=offering&offering_guid=6fa9139cc3584fc0a5662a5c36d68958&widget_guid=78c7ab47bbe04fa59fe0a4889f4cda4c&random=6063894dc432e&iframeid=&mode=p')

xpath_login =  '//*[@id="profile-link-div"]/div/a'
WebDriverWait(driver,30).until(EC.element_to_be_clickable((By.XPATH,xpath_login))) # wait for the element to be available
wait() # to ensure click is not very quick
view1_btn = driver.find_element_by_xpath(xpath_login).click()

wait()
iframe = driver.find_element_by_xpath('//*[@id="rgp00-embedded-modal-frame"]')
driver.switch_to.frame(iframe)

for letter in email:
    inputEmail = driver.find_element_by_name("email")
    inputEmail.send_keys(f'{letter}')
    wait_short()

wait()
for letter in password:
    inputPassword = driver.find_element_by_name("password")
    inputPassword.send_keys(f'{letter}')
    wait_short()

wait()
driver.find_element_by_xpath('/html/body/div[1]/div/div/form/a[2]').click()

wait()
driver.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/a').click()

driver.switch_to.default_content()

In [7]:
# The website we will use as a starting point - I use the north shore booking page (the one with the calendar/booking slots on it)
driver.get('https://app.rockgympro.com/b/widget/?a=offering&offering_guid=6fa9139cc3584fc0a5662a5c36d68958&widget_guid=78c7ab47bbe04fa59fe0a4889f4cda4c&random=6063894dc432e&iframeid=&mode=p')

# Ensure the webpage loads
time.sleep(1)

#_________________________________________ALT method to selecting booking slot
# table = driver.find_elements_by_xpath('//*[@id="offering-page-select-events-table"]/tbody/tr')
# print(table[12])

# booking_slot = driver.find_element_by_xpath(f"//div[contains(., '{time_to_book}')]")
# print(booking_slot)
    
while True:
    try:
        # Set number of members to 1 in the dropdown
        select = Select(driver.find_element_by_id('pcount-pid-1-1307'))
        select.select_by_value(f'{Num_of_members}')

        wait()

        # Determine if booking is in the right month and if not change it
        if booking_month == this_month:
            wait()
        else:
            # Click the arrow on the calendar to go to the next month
            btn_nextmonth = driver.find_element_by_xpath('//*[@id="start_date_calendar"]/div/div/a[2]')
            btn_nextmonth.click()
            wait()

        
        # Chooses the date on the calendar (if the weekday has no button in the first row then go to the second row)
        

        btn_date = driver.find_element_by_xpath(f'//*[@id="start_date_calendar"]/div/table/tbody/tr[{row}]/td[{day_dict[day_to_book]}]/a')
        btn_date.click()

        
        if day_to_book == 'Monday' or day_to_book == 'Tuesday' or day_to_book == 'Friday':
            xpath_slot =  f'//*[@id="offering-page-select-events-table"]/tbody/tr[{time_slot_dict_MTF[time_to_book]}]/td[4]/a'
        elif day_to_book == 'Wednesday' or day_to_book == 'Thursday':
            xpath_slot =  f'//*[@id="offering-page-select-events-table"]/tbody/tr[{time_slot_dict_WTh[time_to_book]}]/td[4]/a'
        elif day_to_book == 'Saturday' or day_to_book == 'Sunday':
            xpath_slot =  f'//*[@id="offering-page-select-events-table"]/tbody/tr[{time_slot_dict_SS[time_to_book]}]/td[4]/a'
        else: print('error 1')

        time.sleep(0.5)
        WebDriverWait(driver,2).until(EC.element_to_be_clickable((By.XPATH,xpath_slot)))
        #wait()
        view1_btn = driver.find_element_by_xpath(xpath_slot).click()
        break
    except:
        driver.refresh()
#_______________________________________________________________________________

# # Selects the booking slot (will change this to select a specific booking variable instead if possible)
# xpath_select =  '//*[@id="offering-page-select-events-table"]/tbody/tr[15]/td[4]/a'
# # wait for the element to ensure it is available
# WebDriverWait(driver,30).until(EC.element_to_be_clickable((By.XPATH,xpath_select)))
# wait() # to ensure click is not very quick
# view1_btn = driver.find_element_by_xpath(xpath_select).click()

# Selects the autofill button for my name which populates my information (must be logged in)
xpath_me =  '//*[@id="theform"]/fieldset[1]/div/div[1]/div/a'
# wait for the element to ensure it is available
WebDriverWait(driver,30).until(EC.element_to_be_clickable((By.XPATH,xpath_me)))
time.sleep(1.3) # to ensure click is not very quick
view1_btn = driver.find_element_by_xpath(xpath_me).click()

wait()
# Confirm the booking is for this location
select = Select(driver.find_element_by_id('booking18edaf725526452baddfcc1f146b196f'))
select.select_by_visible_text('YES')

wait()

# Confirm that you do not have any covid symptoms
select = Select(driver.find_element_by_id('p1959c7777dfd04de8b6d50065ca969b9a'))
select.select_by_visible_text('NO')

wait()

# Click the continue button
xpath_continue =  '//*[@id="theform"]/a[2]'
WebDriverWait(driver,30).until(EC.element_to_be_clickable((By.XPATH,xpath_continue))) # wait for the element to be available
wait() # to ensure click is not very quick
view1_btn = driver.find_element_by_xpath(xpath_continue).click()

# Agree to the terms check box
xpath_agree =  '//*[@id="theform"]/fieldset[5]/div[2]/input'
WebDriverWait(driver,30).until(EC.element_to_be_clickable((By.XPATH,xpath_agree))) # wait for the element to be available
wait() # to ensure click is not very quick
view1_btn = driver.find_element_by_xpath(xpath_agree).click()

# Click on the captcha iframe (clicks on the entire element not just check box)
capt = driver.find_element_by_xpath('//*[@id="google-robot-test"]/div/div[2]/div/div/div/iframe')
action = ActionChains(driver)
action.move_to_element(capt)
action.click()
action.perform() # when using action chains must use the perform button to execute any previously stored actions

wait()

# Switch to the captcha frame and choose the audio captcha option
driver.switch_to.default_content()
WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe[title='recaptcha challenge']")))
wait()
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button#recaptcha-audio-button"))).click()

wait()

# Set a number to break the following loop
sentinel = 1 # this is not a variable dont touch it

# Loop through if there are multiple captchas presented
# Always enter the first captcha and continue to loop until there is no play button to click in the iframe
while sentinel == 1:
    try:
        #switch to recaptcha audio challenge frame
        driver.switch_to.default_content()
        frames= driver.find_elements_by_tag_name("iframe")
        driver.switch_to.frame(frames[-1])
        wait()
        
        #click on the play button
        driver.find_element_by_xpath("/html/body/div/div/div[3]/div/button").click()
        #get the mp3 audio file
        src = driver.find_element_by_id("audio-source").get_attribute("src")
        print("[INFO] Audio src: %s"%src)
        #download the mp3 audio file from the source
        urllib.request.urlretrieve(src, os.path.normpath(os.getcwd()+"\\sample.mp3"))
        wait()
        #load downloaded mp3 audio file as .wav
        #try:
        sound = pydub.AudioSegment.from_mp3(os.path.normpath(os.getcwd()+"\\sample.mp3"))
        sound.export(os.path.normpath(os.getcwd()+"\\sample.wav"), format="wav")
        sample_audio = sr.AudioFile(os.path.normpath(os.getcwd()+"\\sample.wav"))
        #except:
        #    print("[-] Please run program as administrator")

        r= sr.Recognizer()

        with sample_audio as source:
            audio = r.record(source)

        #translate audio to text with google voice recognition
        key=r.recognize_google(audio)
        print("[INFO] Recaptcha Passcode: %s"%key)

        wait()
        #key in results and submit
        #driver.find_element_by_id("audio-response").send_keys(key.lower())
        for letter in key:
            driver.find_element_by_id("audio-response").send_keys(letter)
            wait_short()
        wait()
        driver.find_element_by_id("audio-response").send_keys(Keys.ENTER)
        wait()
    except: sentinel = 0

# Press the 'Complete Booking' Button (** I have commented this out so I can test on the actual website without actually booking lol)
print("now to press complete booking button!")
#xpath_complete =  '//*[@id="confirm_booking_button"]'
#WebDriverWait(driver,30).until(EC.element_to_be_clickable((By.XPATH,xpath_complete))) # wait for the element to be available
#wait() # to ensure click is not very quick
#view1_btn = driver.find_element_by_xpath(xpath_complete).click()

# Text me when the code has run (either the message "you computer or netweork may be sending automated queries" popped up
# or the code actually booked)
# the following line needs your Twilio Account SID and Auth Token
client = Client("ACa3f2f8227c2375850b8a8dd741b7d665", "81e2dedcf51b88ec9031e6e014e5e039")

# change the "from_" number to your Twilio number and the "to" number
# to the phone number you signed up for Twilio with, or upgrade your
# account to send SMS to any phone number
client.messages.create(to="+17789852567", 
                       from_="+12564004669", 
                       body="Your Code Did A Thing and Needs Checking!")

WebDriverException: Message: chrome not reachable
  (Session info: chrome=89.0.4389.128)


In [11]:
if booking_month == this_month:
    print('yep')
else: print('naw')

yep


In [4]:
print(this_day)

Friday


In [ ]:
//*[@id="offering-page-select-events-table"]/tbody/tr[3]/td[4]/a